# Notebook to analyze jumping experiments with eyecams

In [1]:
### import functions

%load_ext autoreload
%autoreload 2

import os, fnmatch
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from glob import glob
import pylab
import analyze_movements as am
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import cm
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib as mpl
from scipy import signal
import matplotlib.colors as mpc
import cv2
import warnings
warnings.filterwarnings('ignore')

### Set general plotting parameters
plt.rcParams.update({'font.size': 15})
plot_params = {'cm':pylab.get_cmap('jet'),
               'pointsize':3,
               'cond_col':['k',[0.6,0.6,0.6]],
               'hmm_col':[[0.6,0.6,0.6], 'k'],
               '4cond_col':[[1,0,0],'k',[0.5,0,0],[0.6,0.6,0.6]],
               'cond_sty':['-',':'],
               '4cond_sty':[':','-',':','-'],
               'plat_cols':['b','r','m']}

save_pdf = False
pp = []
        
print('finished loading dependencies')

finished loading dependencies


In [2]:
### find metadata files and load them into a df
top_pixpercm = np.round(np.sqrt(3025)/3.81,2) #measured square on jumping platform lip 9/11/20
side_pixpercm = np.round(np.sqrt(2500)/3.81,2) #measured square on jumping platform lip 9/11/20
raw_data_path = r'\\goeppert\nlab-nas\Phil\Behavior\Jumping\eyecam_data\eyecam_experiments_out'
data, exptfiles = am.json_to_df(raw_data_path,side_pixpercm,top_pixpercm)

# find DLC files
dlc_path = r'\\goeppert\nlab-nas\Phil\Behavior\Jumping\eyecam_data\reform_clipped_vids_DLC_params'
dlc_files = am.find('*.h5',dlc_path)
dlc_files = [f for f in dlc_files if 'EYE' not in f]
print('%d total DLC trials' % (len(dlc_files)/2))

100%|██████████| 44/44 [00:01<00:00, 41.90it/s]


22 experiments
299 trials
Index(['expdate', 'subject', 'condition', 'platform', 'distance', 'success',
       'laser_trial', 'jump_time', 'trial', 'fps', 'Top_pixpercm',
       'Side_pixpercm', 'ocular', 'Top_Start', 'Top_Jump', 'Top_End',
       'Side_Start', 'Side_Jump', 'Side_End', 'LEYE_Start', 'LEYE_Jump',
       'LEYE_End', 'REYE_Start', 'REYE_Jump', 'REYE_End'],
      dtype='object')
299 total DLC trials


In [3]:
# add DLC data to dataframe
lh_thresh = 0.9
filtered = False
df_all = am.load_dlc_h5_decision_period_only(data,['Side','Top'],dlc_path,lh_thresh,filtered)
df = df_all[['expdate','subject','condition','platform','distance','jumpdist','success','laser_trial','jump_time','trial','fps',
             'Top_pixpercm','Side_pixpercm','ocular','Side_Start','Side_Jump','Top_Start','Top_Jump',
             'Side Nose x','Top Nose x','Side Nose y','Top Nose y','Side Nose likelihood','Top Nose likelihood',
             'Side WiskPad x','Top WiskPad x','Side WiskPad y','Top WiskPad y','Side WiskPad likelihood','Top WiskPad likelihood',
             'Side LEye x','Top LEye x','Side LEye y','Top LEye y','Side LEye likelihood','Top LEye likelihood',
             'Side LEar x','Top LEar x','Side LEar y','Top LEar y','Side LEar likelihood','Top LEar likelihood',
             'Side TakeFL x','Top TakeFL x','Side TakeFL y','Top TakeFL y','Side TakeFL likelihood','Top TakeFL likelihood',
             'Side LandFL x','Top LandFL x','Side LandFL y','Top LandFL y','Side LandFL likelihood','Top LandFL likelihood',
             'Side LandFR x','Top LandFR x','Side LandFR y','Top LandFR y','Side LandFR likelihood','Top LandFR likelihood']].copy()

h5file = r'T:\eyecam_data.h5'
df.to_hdf(h5file,key='df',mode='w')
print('finished saving data!')

trying 071420_PV-CHR11P1LT_eyecams_1
trying 071420_PV-CHR11P1LT_eyecams_2
trying 071420_PV-CHR11P1LT_eyecams_3
trying 071420_PV-CHR11P1LT_eyecams_4
trying 071420_PV-CHR11P1LT_eyecams_5
trying 071420_PV-CHR11P1LT_eyecams_6
trying 071420_PV-CHR11P2LT_eyecams_1
trying 071420_PV-CHR11P2LT_eyecams_2
trying 071420_PV-CHR11P2LT_eyecams_3
trying 071420_PV-CHR11P2LT_eyecams_4
trying 071420_PV-CHR11P2LT_eyecams_5
trying 071420_PV-CHR11P2LT_eyecams_6
trying 071420_PV-CHR11P2LT_eyecams_7
trying 071420_PV-CHR11P2LT_eyecams_8
trying 071420_PV-CHR11P2RT_eyecams_1
trying 071420_PV-CHR11P2RT_eyecams_2
trying 071420_PV-CHR11P2RT_eyecams_3
trying 071520_PV-CHR11P1LT_eyecams_1
trying 071520_PV-CHR11P1LT_eyecams_2
trying 071520_PV-CHR11P1LT_eyecams_3
trying 071520_PV-CHR11P1LT_eyecams_4
trying 071520_PV-CHR11P1LT_eyecams_5
trying 071520_PV-CHR11P1LT_eyecams_6
trying 071520_PV-CHR11P1LT_eyecams_7
trying 071520_PV-CHR11P1LT_eyecams_8
trying 071520_PV-CHR11P1LT_eyecams_9
trying 071520_PV-CHR11P1LT_eyecams_10


In [ ]:
df.keys()

,expdate,subject,condition,platform,distance,jumpdist,success,laser_trial,jump_time,trial,...,Side LEar y,Top LEar y,Side LEar likelihood,Top LEar likelihood,Side TakeFL x,Top TakeFL x,Side TakeFL y,Top TakeFL y,Side TakeFL likelihood,Top TakeFL likelihood
0,071420,PV-CHR11P1LT,eyecams,3,8,8.219056,1,laser off,94,1,...,"[420.994979248047, 362.2344506835939, 303.4285...","[420.994979248047, 362.2344506835939, 303.4285...","[0.9999827146530151, 0.9993003606796265, 0.999...","[0.9999827146530151, 0.9993003606796265, 0.999...","[343.298828125, 457.7956787109375, 572.2246215...","[343.298828125, 457.7956787109375, 572.2246215...","[451.6367802877986, 387.2463750143611, 323.171...","[451.6367802877986, 387.2463750143611, 323.171...","[0.9999967813491821, 0.9999982714653015, 0.999...","[0.9999967813491821, 0.9999982714653015, 0.999..."
1,071420,PV-CHR11P1LT,eyecams,2,12,13.309316,1,laser off,229,2,...,"[419.44985853215104, 362.58957410832295, 305.6...","[419.44985853215104, 362.58957410832295, 305.6...","[0.999971866607666, 0.9997568726539612, 0.9999...","[0.999971866607666, 0.9997568726539612, 0.9999...","[342.68937988281255, 456.98695068359376, 571.2...","[342.68937988281255, 456.98695068359376, 571.2...","[449.2606387679305, 385.45676913902423, 321.65...","[449.2606387679305, 385.45676913902423, 321.65...","[0.9998506307601929, 0.9998537302017212, 0.999...","[0.9998506307601929, 0.9998537302017212, 0.999..."
2,071420,PV-CHR11P1LT,eyecams,2,8,5.484051,1,laser off,303,3,...,"[417.38062956673764, 358.6499532972064, 299.96...","[417.38062956673764, 358.6499532972064, 299.96...","[0.999983549118042, 0.9999922513961792, 0.9999...","[0.999983549118042, 0.9999922513961792, 0.9999...","[343.6437866210938, 458.16572265625007, 572.74...","[343.6437866210938, 458.16572265625007, 572.74...","[448.51901043483196, 384.2447428567069, 319.97...","[448.51901043483196, 384.2447428567069, 319.97...","[0.9999992847442627, 0.9999995231628418, 0.999...","[0.9999992847442627, 0.9999995231628418, 0.999..."
3,071420,PV-CHR11P1LT,eyecams,1,12,12.839377,1,laser off,458,4,...,"[413.86370089661847, 356.2703781427122, 298.69...","[413.86370089661847, 356.2703781427122, 298.69...","[0.9995964765548706, 0.9998345971107483, 0.999...","[0.9995964765548706, 0.9998345971107483, 0.999...","[343.8531372070313, 458.5423095703125, 573.054...","[343.8531372070313, 458.5423095703125, 573.054...","[447.45002800436583, 383.5459874770221, 319.64...","[447.45002800436583, 383.5459874770221, 319.64...","[0.9999935626983643, 0.9999895095825195, 0.999...","[0.9999935626983643, 0.9999895095825195, 0.999..."
4,071420,PV-CHR11P1LT,eyecams,2,16,11.391832,1,laser off,548,5,...,"[413.95961711315954, 354.8567728748783, 295.13...","[413.95961711315954, 354.8567728748783, 295.13...","[0.9999858140945435, 0.9999969005584717, 0.999...","[0.9999858140945435, 0.9999969005584717, 0.999...","[343.3669677734375, 457.82919921875003, 572.22...","[343.3669677734375, 457.82919921875003, 572.22...","[447.25099512648, 383.34362207960504, 319.4362...","[447.25099512648, 383.34362207960504, 319.4362...","[0.999992311000824, 0.9999880790710449, 0.9999...","[0.999992311000824, 0.9999880790710449, 0.9999..."


In [2]:
h5file = r'T:\eyecam_data.h5'
df = pd.read_hdf(h5file)

In [5]:
pdf_file = r'T:\eyecam_plots.pdf'
pp = PdfPages(pdf_file)
save_pdf = True
am.plot_performance(df,'condition',False,5,25,plot_params,save_pdf,pp,suptitle='eyecams')
am.plot_jumpdist(df,'condition',5,25,plot_params,save_pdf,pp,suptitle='eyecams')
pp.close()
print('finished PDF!')

Loading BokehJS ...

finished PDF!


In [ ]:
# function to filter traces
def filter_decision_period(row,side,key,l_thresh,ts_down,st_pt,end_pt):
    #row: dataframe row
    #side: 'Side' or 'Top' for camera
    #key: DLC point, e.g. 'LEye'
    #l_thresh: threshold for likelihood, e.g. 0.95
    #ts_down: downsample factor (usually just 1)
    #st_point: start frame of the output traces (usually use 2)
    #end_pt: end frame of the output traces (usually use -2)
    
    box_sz = 5 #for box filtering
    box = np.ones(box_sz)/box_sz
    
    like = row[side + ' ' + key + ' likelihood']
    xtr = row[side + ' ' + key + ' x']
    ytr = row[side + ' ' + key + ' y']
    
    xtr[like<l_thresh] = np.nan #nan out low likelihoods
    xtr = pd.Series(xtr).interpolate().to_numpy() #interp over nans
    xtr = pd.Series(xtr).fillna(method='bfill').to_numpy() #if first vals are nans fill them
    xtr = signal.medfilt(xtr,kernel_size=3) #median filter
    xtr = np.convolve(xtr, box, mode='same') #smoothing filter
    
    xtr = xtr[:row[side + '_Jump']-row[side + '_Start']]
    xtr = xtr[st_pt:end_pt] #full trace
    xtr = xtr[::ts_down]
    
    ytr[like<l_thresh] = np.nan #nan out low likelihoods
    ytr = pd.Series(ytr).interpolate().to_numpy() #interp over nans
    ytr = pd.Series(ytr).fillna(method='bfill').to_numpy() #if first vals are nans fill them
    ytr = signal.medfilt(ytr,kernel_size=3) #median filter
    ytr = np.convolve(ytr, box, mode='same') #smoothing filter
    
    ytr = ytr[:row[side + '_Jump']-row[side + '_Start']]
    ytr = ytr[st_pt:end_pt] #full trace
    ytr = ytr[::ts_down]
    
    return xtr,ytr